In [1]:
import pandas as pd
from warnings import simplefilter
simplefilter(action="ignore", category=FutureWarning)

In [2]:
df = pd.read_table("potential_mbps.tsv")
no_evid_seqs = set(df['seq_id'])

In [3]:
records = []
species = ['human', 'worm', 'yeast', 'zebrafish']
for s in species:
    df = pd.read_table(f"../metal_type/{s}_pred_mbps_with_type.tsv")
    for seq_id, df_seq in df.groupby(['seq_id']):
        if seq_id not in no_evid_seqs: continue
        
        metal_positions = sorted(df_seq['resi_seq_posi'])
        metals_str = set(df_seq['pred'])
        metals = set()
        for i in metals_str:
            for j in i.split(";"):
                metals.add(j)
        
        records.append({
            "seq_id": seq_id,
            "species": s,
            "pred": ";".join(metals),
            "resi_seq_posi": ";".join([str(i) for i in metal_positions])
        })
df = pd.DataFrame(records)
del records

In [4]:
from itertools import combinations, product
from Bio.SeqRecord import SeqRecord
from Bio import SwissProt
from Bio.Seq import Seq
from pathlib import Path

swiss_file_path = Path("~/database/uniprot/swiss/").expanduser()   
def get_go_term(
    df: pd.DataFrame,
) -> pd.DataFrame:
    
    def gen_swiss_record(source):
        swiss_records = SwissProt.parse(source)
        for swiss_record in swiss_records:
            # Convert the SwissProt record to a SeqRecord
            record = SeqRecord(
                Seq(swiss_record.sequence),
                id=swiss_record.accessions[0],
                name=swiss_record.entry_name,
                description=str(len(swiss_record.references)), # store the number of papers involved
                features=swiss_record.features,
            )
            for cross_reference in swiss_record.cross_references:
                if len(cross_reference) < 2:
                    continue
                database, accession = cross_reference[:2]
                description = cross_reference[2] if len(cross_reference) >= 3 else ""

                dbxref = f"{database}; {accession}; {description}"
                if dbxref not in record.dbxrefs:
                    record.dbxrefs.append(dbxref)
            yield record
    
    species = set(df['species'])
    uniprots = set(df['seq_id'])
    
    records = []
    for s in species:
        swiss_file = swiss_file_path / f"{s}.txt"
        for r in gen_swiss_record(swiss_file):
            if r.id in uniprots:
                seq_len = len(r.seq)
                go_terms = []
                
                for dbxref in r.dbxrefs:
                    database, id, desc = dbxref.split("; ")
                    if database == "GO":
                        go_terms.append(desc)
                    
                records.append({
                    "seq_id": r.id,
                    "seq": str(r.seq),
                    "length": seq_len,
                    "go": ";".join(go_terms),
                    "name": r.name,
                    "cnt_citation": int(r.description),
                })
                
    df_go = pd.DataFrame(records)
    
    return pd.merge(df, df_go, how='outer')


from typing import List
from Bio.PDB.PDBParser import PDBParser
from Bio.PDB.Residue import Residue
from Bio.SeqUtils import seq1
import networkx as nx

def has_suitable_site(
    pdb_file: str,
    positions: List[int],
    coordinate_radius: float = 5.
):
    
    def find_ring(g: nx.Graph) -> nx.Graph:
        ''' Remove leaf nodes from a graph '''
        g_ = g.copy()
        while True:
            leaf_nodes = [n for n, degree in g_.degree() if degree < 2]
            if len(leaf_nodes) == 0:
                break
            else:
                g_.remove_nodes_from(leaf_nodes)
        return g_
    
    def get_hetero_atoms(resi: Residue):
        res_one_letter = seq1(resi.get_resname())
        if res_one_letter == "C":
            return [resi["SG"]]
        elif res_one_letter == "H":
            return [resi["ND1"], resi["NE2"]]
        elif res_one_letter == "E":
            return [resi["OE1"], resi["OE2"]]
        else:
            return [resi["OD1"], resi["OD2"]]
    
    def calc_distance_of_hetero_atoms(resi1: Residue, resi2: Residue):
        try:
            atoms1 = get_hetero_atoms(resi1)
            atoms2 = get_hetero_atoms(resi2)
            min_distance = 100000 # big snumber
            for i, j in product(atoms1, atoms2):
                d = i - j
                if d < min_distance:
                    min_distance = d
            return min_distance
        except:
            return 100000
    
    try:
        residues = list(PDBParser(QUIET=True).get_structure("", pdb_file).get_residues())
    except:
        return
    
    target_residues = []
    has_non_aligned_seq = False
    for p in positions:
        r: Residue = residues[p]
        if seq1(r.get_resname()) in {"C", "H", "E", "D"}:
            target_residues.append(r)
        else:
            has_non_aligned_seq = True
            break
    if has_non_aligned_seq: return False
    
    g = nx.Graph()
    for i, j in combinations(target_residues, 2):
        d = calc_distance_of_hetero_atoms(i, j)
        if d < 2 * coordinate_radius:
            g.add_edge(i.get_full_id(), j.get_full_id())
    
    has_adjacent_residues = False
    g = find_ring(g)
    if len(g.nodes) >= 3:
        has_adjacent_residues = True
    return has_adjacent_residues

from Bio.SeqUtils.ProtParam import ProteinAnalysis
def calc_protein_param(seq: str):
    
    pa = ProteinAnalysis(seq)
    return pa.gravy()

### may be important proteins with high confidence

In [7]:
df_exp = df[df['species'].map(lambda x: x == "human" or x == 'yeast')] # species: human and yeast
df_exp = df_exp[df_exp["resi_seq_posi"].map(lambda x: len(x.split(";")) >= 3)] # pred resi: at least 3

df_exp = get_go_term(df_exp).drop(columns=['seq'])

# protein name in the entry name
df_exp = df_exp[df_exp.apply(lambda row: row['seq_id'] != row['name'].split("_")[0], axis=1)]

# no "ion binding" in go terms (protein level)
df_exp[df_exp['go'].map(lambda x: "ion binding" not in x)].sort_values(by=['cnt_citation'], ascending=False).head() 

,seq_id,species,pred,resi_seq_posi,length,go,name,cnt_citation
136,Q14457,human,ZN,17;20;136;139,450,C:autophagosome;C:cytoplasm;C:cytosol;C:dendri...,BECN1_HUMAN,54
299,Q8NFG4,human,ZN,10;13;81;84,579,C:centrosome;C:cilium;C:cytoplasm;C:cytosol;C:...,FLCN_HUMAN,45
104,P23497,human,ZN,487;489;492,879,"C:chromosome, telomeric region;C:cytoplasm;C:n...",SP100_HUMAN,41
322,Q92597,human,ZN,62;92;94,394,C:adherens junction;C:centrosome;C:cytoplasm;C...,NDRG1_HUMAN,33
470,P14164,yeast,ZN,48;65;70;337;339,731,C:nucleotide-excision repair factor 4 complex;...,ABF1_YEAST,31


### to be validated (Zn)

In [8]:
# species: human and yeast
df_exp = df[df['species'].map(lambda x: x == "human" or x == 'yeast')]

# pred resi: at least 3
df_exp = df_exp[df_exp["resi_seq_posi"].map(lambda x: len(x.split(";")) >= 3)]

# no "membrane" or "zinc" in go terms
df_exp = get_go_term(df_exp)
df_exp = df_exp[df_exp["go"].map(lambda x: "membrane" not in x and "zinc" not in x)]

# less than 300 amino acids
df_exp = df_exp[df_exp["length"].map(lambda x: x < 300)]

# has a suitable coordinated site (distance of hetero atoms no more than 6 angstrom)
af2_pdb_file_path = Path("~/database/pdb/species/").expanduser()
df_exp['struct_suitable'] = df_exp.apply(
    lambda row: has_suitable_site(
            pdb_file = af2_pdb_file_path / f"{row['species']}_af2" / f"AF-{row['seq_id']}-F1-model_v4.pdb",
            positions = [int(i) for i in row['resi_seq_posi'].split(";")],
            coordinate_radius=3
        ), axis=1
)
df_exp = df_exp[df_exp['struct_suitable'].apply(lambda x: x == True)]

# soluble
df_exp['gravy'] = df_exp['seq'].apply(lambda x: ProteinAnalysis(x).gravy())
df_exp['stability'] = df_exp['seq'].apply(lambda x: ProteinAnalysis(x).instability_index())
df_exp = df_exp[df_exp.apply(lambda row: row['gravy'] < 0 and row['stability'] < 40, axis=1)]

df_exp.drop(columns=['seq']) # P40080, Q96MD7

,seq_id,species,pred,resi_seq_posi,length,go,name,cnt_citation,struct_suitable,gravy,stability
49,B3KNS4,human,ZN,4;6;11;13,109,,B3KNS4_HUMAN,5,True,-0.529358,32.041284
57,H3BPM6,human,ZN,14;15;28;31,223,,MKROS_HUMAN,1,True,-0.526906,38.599148
59,H3BU10,human,ZN,137;140;151;154,264,,H3BU10_HUMAN,4,True,-0.150000,31.017803
95,P0DJI9,human,CA;MG,50;54;80;96,122,C:extracellular exosome;C:high-density lipopro...,SAA2_HUMAN,11,True,-0.547541,29.256557
152,Q32NC0,human,ZN,42;45;103;106,220,,CR021_HUMAN,4,True,-0.747273,32.486818
349,Q96MD7,human,ZN,44;47;69;72;85;88;95;98,179,,CI085_HUMAN,6,True,-0.400559,39.160335
365,Q99470,human,ZN;CA;MN,103;127;130,211,C:chaperone complex;C:endoplasmic reticulum;F:...,SDF2_HUMAN,4,True,-0.362085,16.892417
399,Q9H7C9,human,ZN;MN,24;44;118;121,122,C:cytoplasm;P:positive regulation of fat cell ...,AAMDC_HUMAN,8,True,-0.395902,29.971311
518,P39721,yeast,ZN,49;129;153;211,246,C:cytoplasm;F:hydrolase activity,AIM2_YEAST,5,True,-0.081301,37.419919
519,P40080,yeast,ZN,17;20;40;43,203,C:cytoplasm;C:endosome;F:ATPase activator acti...,VFA1_YEAST,5,True,-0.879310,34.606404


### to be presented (SF4 and Ca)

In [7]:
df[df['pred'].map(lambda x: "SF4" in x)] # Q5VUE5

,seq_id,species,pred,resi_seq_posi
399,Q5VUE5,human,SF4;ZN,108;114;115;119;121;122
888,Q9UJT9,human,SF4;ZN,400;426;452
1196,H2KZ52,worm,SF4;ZN,159;165;166;170;173
2663,A0A8M1NGW3,zebrafish,SF4;ZN,18;28;39;49
3078,A0A8M3B2I2,zebrafish,SF4;ZN,102;236;248
3732,A5PL75,zebrafish,SF4;ZN,94;100;101;105;107;108
4011,Q5U3S9,zebrafish,SF4;ZN,54;57;63
4127,Q7T2F0,zebrafish,SF4,6;31
4194,Q9YHT4,zebrafish,SF4,7;31


In [8]:
df[df.apply(lambda row: row['species'] == 'worm' and row['pred'] == "CA" and len(row['resi_seq_posi'].split(";")) >= 4, axis=1)] # Q9XWL1

,seq_id,species,pred,resi_seq_posi
945,A0A0K2WPR7,worm,CA,192;194;197;239
955,A0A0M7RF59,worm,CA,70;135;136;194
976,A0A4V0INN0,worm,CA,608;610;614;655
980,A0A5K1IB90,worm,CA,46;48;56;82;83
1002,A3FPK9,worm,CA,40;42;53;55;56;77
...,...,...,...,...
2276,Q9XUU2,worm,CA,352;361;364;365
2311,Q9XWI4,worm,CA,58;118;119;175;177;183;184
2313,Q9XWL1,worm,CA,45;47;50;53;74
2324,Q9XX41,worm,CA,52;59;60;120;121;174;182;183


In [9]:
df[df.apply(lambda row: row['pred'] == "FE" and len(row['resi_seq_posi'].split(";")) >= 3, axis=1)] # P25362

,seq_id,species,pred,resi_seq_posi
2104,Q9BL09,worm,FE,92;111;112
2378,P25362,yeast,FE,44;82;142;169;204
2549,Q06490,yeast,FE,396;433;488;560
4001,Q5PRC6,zebrafish,FE,260;330;523


In [10]:
df[df.apply(lambda row: row['pred'] == "MN" and len(row['resi_seq_posi'].split(";")) >= 3, axis=1)] # O45841

,seq_id,species,pred,resi_seq_posi
426,Q6NT04,human,MN,177;295;300
1260,O02232,worm,MN,137;139;204;289
1421,O45841,worm,MN,105;107;173;256;273
1427,O45951,worm,MN,176;244;288
1481,O76444,worm,MN,146;148;214;300
1482,O76446,worm,MN,160;162;228;313
1661,Q18357,worm,MN,92;104;106;193
1756,Q20177,worm,MN,25;116;302
2043,Q93570,worm,MN,133;135;203;288
2236,Q9U2B4,worm,MN,128;169;181


In [11]:
df[df.apply(lambda row: row['pred'] == "MG" and len(row['resi_seq_posi'].split(";")) >= 3, axis=1)] # A4QP92

,seq_id,species,pred,resi_seq_posi
189,P0DJI8,human,MG,54;80;96
211,P18405,human,MG,168;201;204
356,Q5BKX5,human,MG,217;219;233
383,Q5T5J6,human,MG,392;484;509
491,Q7L9B9,human,MG,300;447;489
674,Q96EK7,human,MG,43;159;178;180
836,Q9NRR6,human,MG,423;476;555
918,Q9Y2V0,human,MG,165;182;195;213
987,A0A679L8Q3,worm,MG,255;257;294;308
1307,O16723,worm,MG,230;232;271;285
